In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [23]:
## Read The dataset
training = spark.read.csv('test4.csv',header=True,inferSchema=True)

In [24]:
training.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansh| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [25]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [26]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

[Age,Experience] ----> new feature ----> independent feature

In [27]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [28]:
 output = featureassembler.transform(training)

In [29]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|   Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhansh| 30|         8| 25000|          [30.0,8.0]|
|   Sunny| 29|         4| 20000|          [29.0,4.0]|
|    Paul| 24|         3| 20000|          [24.0,3.0]|
|  Harsha| 21|         1| 15000|          [21.0,1.0]|
| Shubham| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [30]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [31]:
finalized_data = output.select("Independent Features","Salary")

In [32]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [33]:
from pyspark.ml.regression import LinearRegression
## Train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [34]:
### Coefficients
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [35]:
### Intercepts
regressor.intercept

16079.136690647425

In [36]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [37]:
pred_results.predictions.show()

C:\Users\Igord\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17214.09079632846|
+--------------------+------+-----------------+



In [38]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(785.909203671541, 617653.2764156357)